In [1]:
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

X, y = make_circles(n_samples=1000, factor=0.3, noise=0.05, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

In [4]:
class KernelPCA():
    def __init__(self, n_components=2, gamma=10):
        self.n_components = n_components
        self.gamma = gamma
    
    def _center_kernel(self, K1, K2):
        return K1 - K1.mean(dim=1, keepdims=True) - K2.mean(dim=0, keepdims=True) + K2.mean()
    
    def fit(self, data):
        self.data = data.clone()
        distances = torch.cdist(data, data) ** 2
        K = torch.exp(-self.gamma * distances)
        K = self._center_kernel(K, K)
        self.K = K
        
        eigval, eigvec = torch.linalg.eigh(K)
        eigval, idx = torch.sort(eigval, descending=True)
        eigvec = eigvec[:, idx]
        self.eigval = eigval[:self.n_components]
        self.eigvec = eigvec[:, :self.n_components]
    
    def transform(self, data):
        distances = torch.cdist(data, self.data) ** 2
        K_test = torch.exp(-self.gamma * distances)
        K_test = self._center_kernel(K_test, self.K)
        scaled_alphas = self.eigvec / torch.sqrt(self.eigval)
        return K_test.mm(scaled_alphas)

In [5]:
import torch

In [6]:
train_data = torch.from_numpy(X_train)
test_data = torch.from_numpy(X_test)

In [8]:
clf_kpca = KernelPCA()
clf_kpca.fit(train_data)
proj_kpca = clf_kpca.transform(test_data)